In [1]:
%matplotlib inline
from src.connect4 import train

import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

from sklearn.model_selection import train_test_split


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)
np.random.seed(0)

In [2]:
channels = 2
filters = 64

n_epocs = 10000
batch_size = 512
test_size = 0.2
learning_rate = 0.0002 * (batch_size / 1024.0)
momentum = 0.0

In [3]:
class Connect4Dataset(data.Dataset):
    def __init__(self, boards, values):
        assert len(boards) == len(values)
        self.boards = boards
        self.values = values
        
    def __len__(self):
        return len(self.boards)
    
    def __getitem__(self, idx):
        return self.boards[idx], self.values[idx]

boards = torch.load('/home/richard/Downloads/connect4_boards.pth').numpy()
values = torch.load('/home/richard/Downloads/connect4_values.pth').numpy()

# Here we don't want to have the player to move channel
boards = boards[:, 3 - channels:]

board_train, board_test, value_train, value_test = train_test_split(boards, values, test_size=test_size, shuffle=True)

In [4]:
# augment data
v_wins = value_train[value_train == 1.0]
v_draws = value_train[value_train == 0.5]
v_losses = value_train[value_train == 0.0]
b_wins = board_train[value_train == 1.0]
b_draws = board_train[value_train == 0.5]
b_losses = board_train[value_train == 0.0]

n_wins = len(v_wins)
n_draws = len(v_draws)
n_losses = len(v_losses)

v_augmented_draws = np.repeat(v_draws, n_wins/n_draws)
v_augmented_losses = np.repeat(v_losses, n_wins/n_losses)
b_augmented_draws = np.repeat(b_draws, n_wins/n_draws, axis=0)
b_augmented_losses = np.repeat(b_losses, n_wins/n_losses, axis=0)

extra_draw_idx = np.random.choice(range(len(v_draws)), n_wins - len(v_augmented_draws), replace=False)
extra_losses_idx = np.random.choice(range(len(v_losses)), n_wins - len(v_augmented_losses), replace=False)

v_augmented_draws = np.hstack([v_augmented_draws, v_draws[extra_draw_idx]])
v_augmented_losses = np.hstack([v_augmented_losses, v_losses[extra_losses_idx]])
b_augmented_draws = np.concatenate([b_augmented_draws, b_draws[extra_draw_idx]], axis=0)
b_augmented_losses = np.concatenate([b_augmented_losses, b_losses[extra_losses_idx]], axis=0)

value_train = np.hstack([v_wins, v_augmented_draws, v_augmented_losses])
board_train = np.concatenate([b_wins, b_augmented_draws, b_augmented_losses], axis=0)

# np.random.shuffle(value_train)
# np.random.shuffle(board_train)

print(len(value_train), len(board_train))

train = Connect4Dataset(torch.from_numpy(board_train), torch.from_numpy(value_train))
test = Connect4Dataset(torch.from_numpy(board_test), torch.from_numpy(value_test))

train_gen = data.DataLoader(train, batch_size, shuffle=False)
test_gen = data.DataLoader(test, batch_size, shuffle=False)

106617 106617


In [5]:
from src.connect4.utils import Connect4Stats as info

# Input with N * channels * (6,7)
# Output with N * filters * (6,7)
convolutional_layer = \
    nn.Sequential(nn.Conv2d(in_channels=channels,
                            out_channels=filters,
                            kernel_size=3,
                            stride=1,
                            padding=1,
                            dilation=1,
                            groups=1,
                            bias=False),
                  nn.BatchNorm2d(filters),
                  nn.LeakyReLU())

# Input with N * filters * (6,7)
# Output with N * filters * (6,7)
class ResidualLayer(nn.Module):
    def __init__(self):
        super(ResidualLayer, self).__init__()
        self.conv1 = nn.Conv2d(filters, filters, 3, padding=1, bias=False)
        self.conv2 = nn.Conv2d(filters, filters, 3, padding=1, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(filters)
        self.batch_norm2 = nn.BatchNorm2d(filters)
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)

        out += residual
        out = self.relu(out)
        return out


# Input with N * filters * (6,7)
# Output with N * 1 * 1
class ValueHead(nn.Module):
    def __init__(self):
        super(ValueHead, self).__init__()
        self.conv1 = nn.Conv2d(filters, 1, 1)
        self.batch_norm = nn.BatchNorm2d(1)
        self.relu = nn.LeakyReLU()
        self.fcN = nn.Sequential(*[nn.Linear(info.area, info.area) for _ in range(4)])
        self.fc2 = nn.Linear(info.area, 1)
        self.tanh = torch.nn.Tanh()
        self.w1 = nn.Parameter(torch.tensor(1.0), requires_grad=False)
        self.w2 = nn.Parameter(torch.tensor(0.5), requires_grad=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = x.view(x.shape[0], 1, -1)
        x = self.fcN(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.tanh(x)
#         map from [-1, 1] to [0, 1]
        x = (x + self.w1) * self.w2
        x = x.view(-1, 1)
        return x

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.002)
    elif type(m) == nn.Conv2d:
        nn.init.normal_(m.weight, std=0.002)
    elif type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight, std=0.002)
#         nn.init.constant_(m.bias, 0)

net = nn.Sequential(convolutional_layer,
                    nn.Sequential(*[ResidualLayer() for _ in range(4)]),
                    ValueHead())

# net.apply(init_normal)

# net = convolutional_layer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (1): Sequential(
    (0): ResidualLayer(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (1): ResidualLayer(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0

In [6]:
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [7]:
def categorise_predictions(preds):
    preds = preds * 3.0
    torch.floor_(preds)
    preds = preds / 2.0
    return preds

class Stats():
    def __init__(self):
        self.n = 0
        self.average_value = 0.0
        self.loss = 0.0
        self.smallest = 1.0
        self.largest = 0.0
        self.correct = {i : 0 for i in [0.0, 0.5, 1.0]}
        self.total = {i : 0 for i in [0.0, 0.5, 1.0]}
        
    def __repr__(self):
        x = "Average loss:  " + "{:.5f}".format(self.loss / self.n) + \
            ",  Smallest:  " + "{:.5f}".format(self.smallest) + \
            ",  Largest:  " + "{:.5f}".format(self.largest) + \
            ",  Average:  " + "{:.5f}".format(self.average_value / self.n)
        
        for k in self.correct:
            x += "\nCategory, # Predictions, # Correct:  {}, {}, {}".format(
                k,
                self.total[k],
                self.correct[k])
        return x

def update_stats(stats: Stats, outputs, values, loss):
    stats.n += len(values)
    stats.average_value += outputs.sum().item()
    stats.loss += loss.item() * len(values)
    stats.smallest = min(stats.smallest, torch.min(outputs).item())
    stats.largest = max(stats.largest, torch.max(outputs).item())
            
    categories = categorise_predictions(outputs)
    values = values.view(-1)
    categories = categories.view(-1)

    for k in stats.correct:
        idx = (values == k).nonzero()
        stats.total[k] += len(idx)
        stats.correct[k] += len(torch.eq(categories[idx], values[idx]).nonzero()) 

    return stats


In [8]:
%%time

for epoch in range(n_epocs):
    
    net = net.train()
    train_stats = Stats()
    test_stats = Stats()
    
    for board, value in train_gen:
        board, value = board.to(device), value.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = net(board)
        loss = criterion(output, value)
        loss.backward()
        optimizer.step()
        
        train_stats = update_stats(train_stats, output, value, loss)

    # validate
    with torch.set_grad_enabled(False):
        net = net.eval()
        for board, value in test_gen:
            board, value = board.to(device), value.to(device)

            output = net(board)
            loss = criterion(output, value)
            
            test_stats = update_stats(test_stats, output, value, loss)
            
    print("Epoch:  ", epoch, "  Train:\n", train_stats, "\nTest:\n", test_stats)
            

print('Finished Training')

Train:
 Average loss:  0.34150,  Smallest:  0.42234,  Largest:  0.55620,  Average:  0.47951
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.46389,  Smallest:  0.42018,  Largest:  0.53730,  Average:  0.47834
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Average loss:  0.34031,  Smallest:  0.42603,  Largest:  0.56021,  Average:  0.48333
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.46215,  Smallest:  0.42355,  Largest:  0.54104,  Average:  0.48186
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Aver

Train:
 Average loss:  0.33581,  Smallest:  0.44390,  Largest:  0.56908,  Average:  0.49769
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45586,  Smallest:  0.44177,  Largest:  0.54882,  Average:  0.49513
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Average loss:  0.33573,  Smallest:  0.44414,  Largest:  0.56880,  Average:  0.49772
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45583,  Smallest:  0.44200,  Largest:  0.54872,  Average:  0.49519
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Aver

Train:
 Average loss:  0.33458,  Smallest:  0.44719,  Largest:  0.56480,  Average:  0.49800
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45544,  Smallest:  0.44480,  Largest:  0.54772,  Average:  0.49593
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Average loss:  0.33452,  Smallest:  0.44737,  Largest:  0.56458,  Average:  0.49801
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45543,  Smallest:  0.44493,  Largest:  0.54771,  Average:  0.49596
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Aver

Train:
 Average loss:  0.33373,  Smallest:  0.44936,  Largest:  0.56161,  Average:  0.49810
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45522,  Smallest:  0.44665,  Largest:  0.54833,  Average:  0.49642
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Average loss:  0.33368,  Smallest:  0.44944,  Largest:  0.56144,  Average:  0.49810
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45522,  Smallest:  0.44672,  Largest:  0.54839,  Average:  0.49644
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Aver

Train:
 Average loss:  0.33304,  Smallest:  0.44761,  Largest:  0.55954,  Average:  0.49808
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45510,  Smallest:  0.44742,  Largest:  0.54994,  Average:  0.49677
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Average loss:  0.33300,  Smallest:  0.44745,  Largest:  0.55947,  Average:  0.49809
Category, # Predictions, # Correct:  0.0, 35539, 0
Category, # Predictions, # Correct:  0.5, 35539, 35539
Category, # Predictions, # Correct:  1.0, 35539, 0 
Test:
 Average loss:  0.45510,  Smallest:  0.44745,  Largest:  0.55007,  Average:  0.49679
Category, # Predictions, # Correct:  0.0, 3300, 0
Category, # Predictions, # Correct:  0.5, 1278, 1278
Category, # Predictions, # Correct:  1.0, 8934, 0
Train:
 Aver

...

Train:
 Average loss:  0.03387,  Smallest:  0.00062,  Largest:  0.99946,  Average:  0.50262
Category, # Predictions, # Correct:  0.0, 35539, 35501
Category, # Predictions, # Correct:  0.5, 35539, 32491
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69476,  Smallest:  0.00096,  Largest:  0.79804,  Average:  0.02683
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934, 2
Train:
 Average loss:  0.03500,  Smallest:  0.00063,  Largest:  0.99946,  Average:  0.50078
Category, # Predictions, # Correct:  0.0, 35539, 35503
Category, # Predictions, # Correct:  0.5, 35539, 32484
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69488,  Smallest:  0.00095,  Largest:  0.80214,  Average:  0.02659
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934

Train:
 Average loss:  0.03327,  Smallest:  0.00060,  Largest:  0.99948,  Average:  0.50288
Category, # Predictions, # Correct:  0.0, 35539, 35505
Category, # Predictions, # Correct:  0.5, 35539, 32639
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69528,  Smallest:  0.00089,  Largest:  0.80022,  Average:  0.02580
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934, 2
Train:
 Average loss:  0.03095,  Smallest:  0.00059,  Largest:  0.99950,  Average:  0.50279
Category, # Predictions, # Correct:  0.0, 35539, 35503
Category, # Predictions, # Correct:  0.5, 35539, 33054
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69537,  Smallest:  0.00087,  Largest:  0.79782,  Average:  0.02562
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934

Train:
 Average loss:  0.03290,  Smallest:  0.00055,  Largest:  0.99952,  Average:  0.50158
Category, # Predictions, # Correct:  0.0, 35539, 35499
Category, # Predictions, # Correct:  0.5, 35539, 33025
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69527,  Smallest:  0.00086,  Largest:  0.78775,  Average:  0.02582
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934, 2
Train:
 Average loss:  0.03282,  Smallest:  0.00056,  Largest:  0.99953,  Average:  0.50264
Category, # Predictions, # Correct:  0.0, 35539, 35503
Category, # Predictions, # Correct:  0.5, 35539, 32852
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69539,  Smallest:  0.00087,  Largest:  0.79355,  Average:  0.02560
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934

Train:
 Average loss:  0.03194,  Smallest:  0.00053,  Largest:  0.99955,  Average:  0.50046
Category, # Predictions, # Correct:  0.0, 35539, 35499
Category, # Predictions, # Correct:  0.5, 35539, 33179
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69558,  Smallest:  0.00085,  Largest:  0.79780,  Average:  0.02522
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934, 2
Train:
 Average loss:  0.03189,  Smallest:  0.00054,  Largest:  0.99955,  Average:  0.50370
Category, # Predictions, # Correct:  0.0, 35539, 35499
Category, # Predictions, # Correct:  0.5, 35539, 32958
Category, # Predictions, # Correct:  1.0, 35539, 35539 
Test:
 Average loss:  0.69549,  Smallest:  0.00086,  Largest:  0.79534,  Average:  0.02539
Category, # Predictions, # Correct:  0.0, 3300, 3300
Category, # Predictions, # Correct:  0.5, 1278, 0
Category, # Predictions, # Correct:  1.0, 8934

KeyboardInterrupt: 

In [9]:
# validate
with torch.set_grad_enabled(False):
    net = net.eval()
    correct = {i : 0 for i in [0, 0.5, 1]}
    total = {i : 0 for i in [0, 0.5, 1]}
    for board, value in test_gen:
        board, value = board.to(device), value.to(device)

        outputs = net(board)
        categories = categorise_predictions(outputs)

        for k in correct:
            idx = (categories == k).nonzero()
            total[k] += len(idx)
            correct[k] += (categories[idx] == value[idx]).nonzero().sum().item()
    for k in correct:
        print('Category, # Predictions, Accuracy of the network on the test%s: %d %d %%' % (
            k,
            total[k],
            (100 * float(correct[k]) / float(total[k])) if total[k] != 0 else 0.))

RuntimeError: The size of tensor a (2) must match the size of tensor b (0) at non-singleton dimension 1

In [ ]:
assert(False)
# save that crap
torch.save({
    'net_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
},
    open('/home/richard/Downloads/nn.pth', 'wb'))

In [ ]:
# alternatively load it
assert(False)
checkpoint = torch.load('/home/richard/Downloads/nn.pth')
net.load_state_dict(checkpoint['net_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))